In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import pandas_datareader.data as web
import time
import tensorflow
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import logging
import math
import os
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation
from keras import optimizers
from sklearn.metrics import mean_squared_error
from keras.models import load_model

Using TensorFlow backend.


In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

In [3]:
total = pd.read_excel('./data/corr/total_재무지표.xlsx')

In [4]:
total = total[total.columns[:374]]

In [5]:
nc = total[total['회사명'] == '(주)엔씨소프트']
sdi = total[total['회사명'] == '삼성SDI(주)']
ssem = total[total['회사명'] == '삼성전기(주)']
hynix = total[total['회사명'] == '에스케이하이닉스(주)']

# 재무지표 로드

In [6]:
nc = nc.dropna(axis = 1)
sdi = sdi.dropna(axis = 1)
ssem = ssem.dropna(axis = 1)
hynix = hynix.dropna(axis = 1)

In [7]:
nc.sort_values('회계년도', ascending= True, inplace = True)
nc.reset_index(drop = True, inplace = True)
nc = nc.loc[5:,:]
nc.reset_index(drop = True, inplace = True)

In [8]:
sdi.sort_values('회계년도', ascending= True, inplace = True)
sdi.reset_index(drop = True, inplace = True)
sdi = sdi.loc[2:,:]
sdi.reset_index(drop = True, inplace = True)

In [9]:
ssem.sort_values('회계년도', ascending= True, inplace = True)
ssem.reset_index(drop = True, inplace = True)
ssem = ssem.loc[2:,:]
ssem.reset_index(drop = True, inplace = True)

In [10]:
hynix.sort_values('회계년도', ascending= True, inplace = True)
hynix.reset_index(drop = True, inplace = True)
hynix = hynix.loc[3:,:]
hynix.reset_index(drop = True, inplace = True)

# 재무비율 로드

In [11]:
nc_ratio = pd.read_excel('./data/corr/nc_total.xlsx')
sdi_ratio = pd.read_excel('./data/corr/sdi_total.xlsx')
ssem_ratio = pd.read_excel('./data/corr/삼성전기_total.xlsx')
hynix_ratio = pd.read_excel('./data/corr/하이닉스_total.xlsx')

In [12]:
nc_ratio = nc_ratio.dropna(axis = 1)
sdi_ratio = sdi_ratio.dropna(axis = 1)
ssem_ratio = ssem_ratio.dropna(axis = 1)
hynix_ratio = hynix_ratio.dropna(axis = 1)

In [13]:
nc_ratio.corr()['종가'].sort_values(ascending = False)

종가                               1.000000
산술평균                             0.998191
최저가                              0.995621
최고가                              0.993931
[공통]유보율(IFRS연결)                  0.917111
거래년도(*)                          0.821830
[공통]매출액증가율(IFRS연결)               0.703820
[공통]1주당매출액(IFRS연결)               0.625445
[공통]1주당정상영업이익(IFRS연결)            0.613842
[공통]차입금의존도(IFRS연결)               0.607227
[공통]차입금비율(IFRS연결)                0.605129
[공통]부가가치(IFRS연결)                 0.578461
[공통]자본금회전률(IFRS연결)               0.569232
[공통]자본금정상영업이익률(IFRS연결)           0.560964
거래대금                             0.556548
[공통]총자본증가율(IFRS연결)               0.551842
[공통]1주당순이익(IFRS연결)               0.525234
[공통]자기자본증가율(IFRS연결)              0.504292
[공통]자본금순이익률(IFRS연결)              0.498383
[공통]총포괄이익증가율(IFRS연결)             0.481992
[공통]1주당 CASH FLOW(IFRS연결)        0.434028
[공통]정상영업이익증가율(IFRS연결)            0.422587
[공통]세금과공과 대 총비용비율(IFRS연결)        0.392516
[공통]순이익증가율(IFRS연결)               0

# 재무지표와 비율 하나의 데이터프레임으로 merge

### cf. 재무비율은 6개월 단위라  지표도 6개월 단위로 계산

In [14]:
nc = pd.merge(nc_ratio, nc, how = 'left', on = ['회사명', '거래소코드','회계년도'])
sdi = pd.merge(sdi_ratio, sdi, how = 'left', on = ['회사명', '거래소코드','회계년도'])
ssem = pd.merge(ssem_ratio, ssem, how = 'left', on = ['회사명', '거래소코드','회계년도'])
hynix = pd.merge(hynix_ratio, hynix, how = 'left', on = ['회사명', '거래소코드','회계년도'])

# 상관관계 만들기

In [15]:
nc_corr = nc.corr(method = 'pearson')
df_nc = pd.DataFrame(nc_corr['종가']).reset_index().sort_values('종가', ascending = False)
df_nc.columns = ['지표','종가']

sdi_corr = sdi.corr(method = 'pearson')
df_sdi = pd.DataFrame(sdi_corr['종가']).reset_index().sort_values('종가', ascending = False)
df_sdi.columns = ['지표','종가']

ssem_corr = ssem.corr(method = 'pearson')
df_ssem = pd.DataFrame(ssem_corr['종가']).reset_index().sort_values('종가', ascending = False)
df_ssem.columns = ['지표','종가']

hynix_corr = hynix.corr(method = 'pearson')
df_hynix = pd.DataFrame(hynix_corr['종가']).reset_index().sort_values('종가', ascending = False)
df_hynix.columns = ['지표','종가']

In [16]:
list_corr = []
order = 15

for i in range(len(df_nc.index[:order])):
    list_corr.append(df_nc.index[:order][i])
    list_corr.append(df_sdi.index[:order][i])
    list_corr.append(df_ssem.index[:order][i])
    list_corr.append(df_hynix.index[:order][i])

variable = pd.Series(list_corr).value_counts()
variable = pd.DataFrame(variable).reset_index()
variable.columns = ['var_index', 'number']

In [17]:
var = []
for i in range(len(variable)):
    var.append(df_nc.reset_index()['지표'][variable.index[i]])
var = pd.DataFrame(var)
var.columns = ['var']

In [18]:
corr = pd.concat([variable,var], axis = 1)
corr = corr[['var_index','var','number']]
corr.head(20) # 52번까지 재무비율

,var_index,var,number
0,46,종가,4
1,44,산술평균,4
2,45,최저가,4
3,47,최고가,4
4,107,매도가능금융자산평가손익(IFRS연결)(천원),3
5,14,기말기타포괄손익누계액(*)(IFRS연결)(천원),2
6,10,기타포괄손익누계액(*)(IFRS연결)(천원),2
7,49,부채(*)(IFRS연결)(천원),2
8,50,[공통]유보율(IFRS연결),2
9,51,자산(*)(IFRS연결)(천원),2


# 주가와 높은 상관관계를 가진 변수간 상관관계 보기

In [20]:
nc_corr[corr['var'][:20]]

,거래소코드,[공통]총자본증가율(IFRS연결),[공통]자기자본증가율(IFRS연결),[공통]매출액증가율(IFRS연결),[공통]정상영업이익증가율(IFRS연결),[공통]순이익증가율(IFRS연결),[공통]총포괄이익증가율(IFRS연결),[공통]매출액정상영업이익률(IFRS연결),[공통]매출액순이익률(IFRS연결),[공통]총자본사업이익률(IFRS연결),[공통]총자본정상영업이익률(IFRS연결),[공통]총자본순이익률(IFRS연결),[공통]자기자본정상영업이익률(IFRS연결),[공통]자기자본순이익률(IFRS연결),[공통]자본금정상영업이익률(IFRS연결),[공통]자본금순이익률(IFRS연결),[공통]영업비용 대 영업수익비율(IFRS연결),[공통]수지비율(관계기업투자손익 제외)(IFRS연결),[공통]세금과공과 대 총비용비율(IFRS연결),[공통]유보율(IFRS연결)
거래소코드,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
[공통]총자본증가율(IFRS연결),NaN,1.000000,0.977752,0.430355,0.056542,-0.010886,0.294808,-0.113490,-0.251740,0.028919,0.046464,-0.019074,0.038403,-0.042505,0.395548,0.346187,0.113490,0.109865,0.033310,0.547468
[공통]자기자본증가율(IFRS연결),NaN,0.977752,1.000000,0.339622,0.036431,-0.034147,0.235737,-0.176594,-0.340574,-0.050416,-0.034132,-0.109482,-0.043386,-0.130937,0.287163,0.226323,0.176594,0.178758,0.048347,0.495024
[공통]매출액증가율(IFRS연결),NaN,0.430355,0.339622,1.000000,0.784055,0.771551,0.124200,0.513026,0.392993,0.334846,0.356801,0.321978,0.347631,0.295902,0.764410,0.753482,-0.513026,-0.514460,0.671096,0.766343
[공통]정상영업이익증가율(IFRS연결),NaN,0.056542,0.036431,0.784055,1.000000,0.990174,-0.417942,0.653949,0.476314,0.187020,0.196712,0.131067,0.181983,0.107087,0.469159,0.428491,-0.653949,-0.644334,0.961016,0.548889
[공통]순이익증가율(IFRS연결),NaN,-0.010886,-0.034147,0.771551,0.990174,1.000000,-0.408223,0.674967,0.540629,0.162581,0.171588,0.130178,0.155826,0.106651,0.439793,0.415599,-0.674967,-0.675810,0.947770,0.522269
[공통]총포괄이익증가율(IFRS연결),NaN,0.294808,0.235737,0.124200,-0.417942,-0.408223,1.000000,-0.324068,-0.203420,0.024337,0.046292,0.094527,0.051574,0.089801,0.254873,0.309752,0.324068,0.336187,-0.494346,0.359264
[공통]매출액정상영업이익률(IFRS연결),NaN,-0.113490,-0.176594,0.513026,0.653949,0.674967,-0.324068,1.000000,0.884845,0.502505,0.522592,0.444720,0.504755,0.412207,0.633185,0.627129,-1.000000,-0.988220,0.491472,0.297979
[공통]매출액순이익률(IFRS연결),NaN,-0.251740,-0.340574,0.392993,0.476314,0.540629,-0.203420,0.884845,1.000000,0.374125,0.387801,0.430179,0.373964,0.408385,0.436654,0.504489,-0.884845,-0.929745,0.278761,0.107485
[공통]총자본사업이익률(IFRS연결),NaN,0.028919,-0.050416,0.334846,0.187020,0.162581,0.024337,0.502505,0.374125,1.000000,0.998786,0.967771,0.998452,0.960695,0.761844,0.778253,-0.502505,-0.477263,-0.000833,-0.035890


# 상관관계 시각화

In [21]:
# import matplotlib.pyplot as plt
# from matplotlib import font_manager, rc
# import seaborn as sns
# import matplotlib
# font_location = "c:/Windows/fonts/malgun.ttf"
# font_name = font_manager.FontProperties(fname=font_location).get_name()
# matplotlib.rc('font', family=font_name)

In [22]:
# plt.figure(figsize=(20,10))

# sns.heatmap(data = nc_corr[corr['var'][:20]], annot = True, cmap = 'bwr')
# # plt.savefig('tmp.png')

# 딥러닝 모델 구축

In [23]:
# 재무피쳐값 입력
corr.loc[1:13,'var']

1          [공통]총자본증가율(IFRS연결)
2         [공통]자기자본증가율(IFRS연결)
3          [공통]매출액증가율(IFRS연결)
4       [공통]정상영업이익증가율(IFRS연결)
5          [공통]순이익증가율(IFRS연결)
6        [공통]총포괄이익증가율(IFRS연결)
7      [공통]매출액정상영업이익률(IFRS연결)
8         [공통]매출액순이익률(IFRS연결)
9        [공통]총자본사업이익률(IFRS연결)
10     [공통]총자본정상영업이익률(IFRS연결)
11        [공통]총자본순이익률(IFRS연결)
12    [공통]자기자본정상영업이익률(IFRS연결)
13       [공통]자기자본순이익률(IFRS연결)
Name: var, dtype: object

In [24]:
# 재무 피쳐값 입력 및 종가 입력
nc_deep = nc[corr.loc[1:13,'var']]

nc_deep['종가'] = nc['종가']

In [25]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import scale, robust_scale, minmax_scale, maxabs_scale

# minmax_scale(nc_deep)
nc_deep = pd.DataFrame(scale(nc_deep))

# 샤비어함수 및 히든레이어 5개

In [47]:
def xavier_init(n_inputs, n_outputs, uniform=True):
    if uniform:
        # 6 was used in the paper.
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        # 3 gives us approximately the same limints as above since this repicks
        # values greater than 2 standard deviations from the mean.
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

In [28]:
# Review : Learning rate and Evaluation
import tensorflow as tf
import random
import matplotlib.pyplot as plt

tf.reset_default_graph()


x_data = nc_deep.iloc[:-1,:-1]
y_data = nc_deep.iloc[:-1,-1] # 괄호를 한번 더 씌어야 한다는 거 주의!!!!!!

print(y_data)
tf.set_random_seed(777)  # reproducibility
placeholder_num = len(x_data.columns)

# parameters
learning_rate = 0.001



X = tf.placeholder(tf.float32, [None, placeholder_num])
Y = tf.placeholder(tf.float32, [None, 1])
keep_prob = tf.placeholder(tf.float32)


W1 = tf.get_variable("W1", shape=[placeholder_num, 32], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([32]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)


W2 = tf.get_variable("W2", shape=[32, 64], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([64]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("W3", shape=[64, 64], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([64]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)


W4 = tf.get_variable("W4", shape=[64, 64], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([64]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)


W5 = tf.get_variable("W5", shape=[64, 1], initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([1]))
L5 = tf.nn.relu(tf.matmul(L4, W5) + b5)

hypothesis = tf.matmul(L4, W5) + b5


# define cost/loss & optimizer
cost = tf.reduce_mean(tf.square(hypothesis - Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)


# correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

          13
0  -1.099452
1  -0.974215
2  -0.032217
3  -0.750967
4  -0.756412
5  -0.576725
6  -0.418817
7  -0.168344
8  -0.043107
9   1.383504
10  2.134925
11  1.301828


In [29]:
# Launch the graph in a session.
sess = tf.Session()

# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

       
for step in tqdm_notebook(range(2000001)):
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train],
                                   feed_dict={X: x_data, Y: y_data, keep_prob: 1.0})

    if step % 10000 == 0 or step < 10 :
        print("\nStep : {} \nCost : {} \nPrediction :\n{}".format(step, cost_val, hy_val))
       


Step : 0 
Cost : 8.540766716003418 
Prediction :
[[-2.7130806]
 [-2.554535 ]
 [-2.6653204]
 [-2.7428622]
 [-2.6207156]
 [-2.9090843]
 [-2.786583 ]
 [-2.6886473]
 [-2.6266527]
 [-3.073636 ]
 [-2.5660932]
 [-2.8179975]]

Step : 1 
Cost : 8.496781349182129 
Prediction :
[[-2.704463 ]
 [-2.545785 ]
 [-2.6568465]
 [-2.734706 ]
 [-2.61222  ]
 [-2.9015188]
 [-2.7785816]
 [-2.6802928]
 [-2.6181746]
 [-3.0658998]
 [-2.558342 ]
 [-2.8103006]]

Step : 2 
Cost : 8.453083992004395 
Prediction :
[[-2.6958756]
 [-2.5370657]
 [-2.6484025]
 [-2.7265787]
 [-2.603753 ]
 [-2.8939795]
 [-2.7706082]
 [-2.671967 ]
 [-2.6097252]
 [-3.0581918]
 [-2.5506177]
 [-2.8026304]]

Step : 3 
Cost : 8.409668922424316 
Prediction :
[[-2.6873183]
 [-2.5283766]
 [-2.639988 ]
 [-2.7184796]
 [-2.5953157]
 [-2.8864665]
 [-2.762663 ]
 [-2.663671 ]
 [-2.601306 ]
 [-3.05051  ]
 [-2.54292  ]
 [-2.7949867]]

Step : 4 
Cost : 8.366538047790527 
Prediction :
[[-2.6787908]
 [-2.519719 ]
 [-2.6316025]
 [-2.7104082]
 [-2.5869086]
 [-2

 [ 1.3202367 ]]

Step : 270000 
Cost : 0.04821543022990227 
Prediction :
[[-0.89411837]
 [-0.78936106]
 [-0.635583  ]
 [-0.5345928 ]
 [-0.5259232 ]
 [-0.5768939 ]
 [-0.4231522 ]
 [-0.23177439]
 [-0.22711694]
 [ 1.3779085 ]
 [ 2.1332355 ]
 [ 1.3197684 ]]

Step : 280000 
Cost : 0.04708734527230263 
Prediction :
[[-0.89882785]
 [-0.78776807]
 [-0.63043934]
 [-0.53826225]
 [-0.5332093 ]
 [-0.5784777 ]
 [-0.41833276]
 [-0.22968185]
 [-0.22476548]
 [ 1.378104  ]
 [ 2.134797  ]
 [ 1.3193619 ]]

Step : 290000 
Cost : 0.045910388231277466 
Prediction :
[[-0.9027341 ]
 [-0.7859196 ]
 [-0.6246968 ]
 [-0.54366356]
 [-0.53974825]
 [-0.57994074]
 [-0.4142931 ]
 [-0.22719961]
 [-0.22201556]
 [ 1.3772533 ]
 [ 2.1369915 ]
 [ 1.3183837 ]]

Step : 300000 
Cost : 0.04475446045398712 
Prediction :
[[-0.9065327 ]
 [-0.78446835]
 [-0.6189473 ]
 [-0.5489639 ]
 [-0.5465445 ]
 [-0.580847  ]
 [-0.4104157 ]
 [-0.22459131]
 [-0.219679  ]
 [ 1.3773704 ]
 [ 2.138719  ]
 [ 1.3175731 ]]

Step : 310000 
Cost : 0.043623

 [ 1.3095584 ]]

Step : 620000 
Cost : 0.019515948370099068 
Prediction :
[[-0.99124056]
 [-0.7776324 ]
 [-0.43842608]
 [-0.710295  ]
 [-0.71289176]
 [-0.58327824]
 [-0.3334278 ]
 [-0.16990185]
 [-0.13066965]
 [ 1.3857632 ]
 [ 2.1496367 ]
 [ 1.3093696 ]]

Step : 630000 
Cost : 0.019002532586455345 
Prediction :
[[-0.9926329 ]
 [-0.7790174 ]
 [-0.43283612]
 [-0.71395177]
 [-0.7163599 ]
 [-0.583265  ]
 [-0.33236486]
 [-0.16878092]
 [-0.12834173]
 [ 1.386096  ]
 [ 2.1491904 ]
 [ 1.3092649 ]]

Step : 640000 
Cost : 0.01849999651312828 
Prediction :
[[-0.9940074 ]
 [-0.780443  ]
 [-0.42724448]
 [-0.7175711 ]
 [-0.7196019 ]
 [-0.58328646]
 [-0.33142143]
 [-0.16792125]
 [-0.12600166]
 [ 1.3862753 ]
 [ 2.149117  ]
 [ 1.3092055 ]]

Step : 650000 
Cost : 0.018007608130574226 
Prediction :
[[-0.9953461 ]
 [-0.7819093 ]
 [-0.4216593 ]
 [-0.7211117 ]
 [-0.72263986]
 [-0.58333194]
 [-0.33059448]
 [-0.16707164]
 [-0.12364686]
 [ 1.3863502 ]
 [ 2.1490068 ]
 [ 1.3090715 ]]

Step : 660000 
Cost : 0.0175

 [ 1.3055129 ]]

Step : 970000 
Cost : 0.006340863183140755 
Prediction :
[[-1.034827  ]
 [-0.8620551 ]
 [-0.25792027]
 [-0.75633806]
 [-0.7636519 ]
 [-0.5948047 ]
 [-0.3366837 ]
 [-0.15716106]
 [-0.074449  ]
 [ 1.3858204 ]
 [ 2.1455507 ]
 [ 1.3053153 ]]

Step : 980000 
Cost : 0.006087970454245806 
Prediction :
[[-1.0360327 ]
 [-0.86455625]
 [-0.25326437]
 [-0.75663644]
 [-0.7638008 ]
 [-0.5947787 ]
 [-0.3378225 ]
 [-0.15713727]
 [-0.07348669]
 [ 1.385818  ]
 [ 2.1453362 ]
 [ 1.3051887 ]]

Step : 990000 
Cost : 0.005842712242156267 
Prediction :
[[-1.0372422 ]
 [-0.8670142 ]
 [-0.24865621]
 [-0.75691134]
 [-0.76392394]
 [-0.59471637]
 [-0.33896762]
 [-0.15716183]
 [-0.07257128]
 [ 1.3858347 ]
 [ 2.1450858 ]
 [ 1.3051443 ]]

Step : 1000000 
Cost : 0.005604434758424759 
Prediction :
[[-1.038444  ]
 [-0.86944085]
 [-0.24409688]
 [-0.7571636 ]
 [-0.764     ]
 [-0.594654  ]
 [-0.34014827]
 [-0.15721864]
 [-0.07167464]
 [ 1.3858631 ]
 [ 2.1448278 ]
 [ 1.3050723 ]]

Step : 1010000 
Cost : 0.0

 [ 1.3041706 ]]

Step : 1320000 
Cost : 0.0011297059245407581 
Prediction :
[[-1.0706098 ]
 [-0.9309706 ]
 [-0.12689859]
 [-0.75315875]
 [-0.75867015]
 [-0.58900505]
 [-0.37984806]
 [-0.16338521]
 [-0.05589879]
 [ 1.3851342 ]
 [ 2.1387305 ]
 [ 1.3040946 ]]

Step : 1330000 
Cost : 0.0010695727542042732 
Prediction :
[[-1.0714214 ]
 [-0.93212396]
 [-0.12432504]
 [-0.7529146 ]
 [-0.75862926]
 [-0.5888048 ]
 [-0.3809052 ]
 [-0.16350263]
 [-0.05563039]
 [ 1.385108  ]
 [ 2.1386912 ]
 [ 1.3039513 ]]

Step : 1340000 
Cost : 0.001012263004668057 
Prediction :
[[-1.0721848 ]
 [-0.93326527]
 [-0.12180531]
 [-0.7526693 ]
 [-0.7585669 ]
 [-0.58855766]
 [-0.3819409 ]
 [-0.1636076 ]
 [-0.0553748 ]
 [ 1.3850527 ]
 [ 2.1386569 ]
 [ 1.3038073 ]]

Step : 1350000 
Cost : 0.0009579745237715542 
Prediction :
[[-1.072921  ]
 [-0.9343881 ]
 [-0.11935896]
 [-0.75242454]
 [-0.7584546 ]
 [-0.5882866 ]
 [-0.38294524]
 [-0.16376561]
 [-0.05513358]
 [ 1.3849688 ]
 [ 2.1386533 ]
 [ 1.3036647 ]]

Step : 1360000 
Cost

 [ 1.3024216 ]]

Step : 1660000 
Cost : 0.00019817262364085764 
Prediction :
[[-1.0865922 ]
 [-0.9573155 ]
 [-0.07195139]
 [-0.7499457 ]
 [-0.75639206]
 [-0.58320075]
 [-0.4024393 ]
 [-0.16667062]
 [-0.04876572]
 [ 1.3840055 ]
 [ 2.1361804 ]
 [ 1.3024316 ]]

Step : 1670000 
Cost : 0.00019028109090868384 
Prediction :
[[-1.0868409 ]
 [-0.9576662 ]
 [-0.07111365]
 [-0.74984616]
 [-0.7564425 ]
 [-0.5831105 ]
 [-0.40270656]
 [-0.16665709]
 [-0.04868704]
 [ 1.3839383 ]
 [ 2.1361003 ]
 [ 1.302403  ]]

Step : 1680000 
Cost : 0.00018282940436620265 
Prediction :
[[-1.0870657 ]
 [-0.95799476]
 [-0.07029617]
 [-0.74976474]
 [-0.75645953]
 [-0.5830408 ]
 [-0.40296537]
 [-0.16667825]
 [-0.04861063]
 [ 1.3838696 ]
 [ 2.136045  ]
 [ 1.302367  ]]

Step : 1690000 
Cost : 0.0001758321886882186 
Prediction :
[[-1.0872746 ]
 [-0.95828897]
 [-0.06951833]
 [-0.7496813 ]
 [-0.7564197 ]
 [-0.5829573 ]
 [-0.40323907]
 [-0.16678298]
 [-0.04853028]
 [ 1.3838077 ]
 [ 2.1360497 ]
 [ 1.3023155 ]]

Step : 1700000 


 [ 1.3020644 ]]

Step : 2000000 
Cost : 6.114699499448761e-05 
Prediction :
[[-1.0927246 ]
 [-0.9646204 ]
 [-0.05401689]
 [-0.7489762 ]
 [-0.7564514 ]
 [-0.5807341 ]
 [-0.4096833 ]
 [-0.16719031]
 [-0.04703885]
 [ 1.3833759 ]
 [ 2.1358614 ]
 [ 1.302011  ]]


실제값
0  -1.099452
1  -0.974215
2  -0.032217
3  -0.750967
4  -0.756412
5  -0.576725
6  -0.418817
7  -0.168344
8  -0.043107
9   1.383504
10  2.134925
11  1.301828

Step : 1 
Cost : 8.496781349182129 
Prediction :
[[-2.704463 ]
 [-2.545785 ]
 [-2.6568465]
 [-2.734706 ]
 [-2.61222  ]
 [-2.9015188]
 [-2.7785816]
 [-2.6802928]
 [-2.6181746]
 [-3.0658998]
 [-2.558342 ]
 [-2.8103006]]
 
Step : 10000 
Cost : 0.7597538828849792 
Prediction :
[[-0.0910393 ]
 [-0.03625989]
 [-0.10393304]
 [-0.1150068 ]
 [ 0.03890622]
 [-0.2667489 ]
 [-0.14521682]
 [-0.10149986]
 [-0.0137009 ]
 [ 0.13332021]
 [ 0.4574176 ]
 [ 0.02707112]]
 
Step : 100000 
Cost : 0.07895835489034653 
Prediction :
[[-0.7541409 ]
 [-0.8597247 ]
 [-0.7162141 ]
 [-0.4943511 ]
 [-0.34879994]
 [-0.45882875]
 [-0.4853145 ]
 [-0.34539455]
 [-0.2862141 ]
 [ 1.3516657 ]
 [ 2.0651321 ]
 [ 1.3272438 ]]
 

Step : 1000000 
Cost : 0.005604434758424759 
Prediction :
[[-1.038444  ]
 [-0.86944085]
 [-0.24409688]
 [-0.7571636 ]
 [-0.764     ]
 [-0.594654  ]
 [-0.34014827]
 [-0.15721864]
 [-0.07167464]
 [ 1.3858631 ]
 [ 2.1448278 ]
 [ 1.3050723 ]]
 

Step : 2000000 
Cost : 6.114699499448761e-05 
Prediction :
[[-1.0927246 ]
 [-0.9646204 ]
 [-0.05401689]
 [-0.7489762 ]
 [-0.7564514 ]
 [-0.5807341 ]
 [-0.4096833 ]
 [-0.16719031]
 [-0.04703885]
 [ 1.3833759 ]
 [ 2.1358614 ]
 [ 1.302011  ]]
 
실제값
0  -1.099452
1  -0.974215
2  -0.032217
3  -0.750967
4  -0.756412
5  -0.576725
6  -0.418817
7  -0.168344
8  -0.043107
9   1.383504
10  2.134925
11  1.301828